In [1]:
import pandas as pd
import numpy as np

#### Extraccion 'homicidios.xlsx'

In [56]:
hoja = 'HECHOS'
df_homicidios = pd.read_excel('Datasets/homicidios.xlsx', sheet_name=hoja)
df_homicidios.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


In [57]:
df_homicidios.shape

(696, 21)

In [58]:
# Esta funcion muestra la cantidad y el porcentaje de valores nulos de cada columna
def datosNulos(df):
    datos = []   
    
    for name in df.columns:# Recorre cada columna del dataframe
        cantidad_nulos = df[name].isnull().sum()
        #Cacula el porcentaje de nulos por columna
        porcentaje = round( (cantidad_nulos / len(df)) * 100)
        tipo_de_dato = df[name].dtypes 
        #Agrega los datos a la lista
        datos.append({
                   'Columna':name,
                   'Tipo de dato' :tipo_de_dato,
                   'Nulos': cantidad_nulos,
                   'Porcentaje': f"{porcentaje} %"  })
    
    return  pd.DataFrame(datos) #Muestra los datos como un dataframe

In [59]:
datosNulos(df_homicidios)

,Columna,Tipo de dato,Nulos,Porcentaje
0,ID,object,0,0 %
1,N_VICTIMAS,int64,0,0 %
2,FECHA,datetime64[ns],0,0 %
3,AAAA,int64,0,0 %
4,MM,int64,0,0 %
5,DD,int64,0,0 %
6,HORA,object,0,0 %
7,HH,object,0,0 %
8,LUGAR_DEL_HECHO,object,0,0 %
9,TIPO_DE_CALLE,object,0,0 %


In [60]:
# Muestra solo las columnas que tienen datos nulos y su porcentaje
def porcentaje_nulos(df):
    porcentaje = round( (df.isnull().sum() / len(df)) * 100 ,2) 
    filtro_porcentaje = porcentaje[porcentaje > 0] 
    
    print("Porcentaje de nulos por columna:")
    return filtro_porcentaje.apply(lambda x: f"{x} %")
    
porcentaje_nulos(df_homicidios)

Porcentaje de nulos por columna:


Calle                     0.14 %
Altura                   81.47 %
Cruce                    24.57 %
Dirección Normalizada     1.15 %
dtype: object

In [61]:
# Muestra las columnas que le hacen falta datos
resultado = df_homicidios.isin(['SD','SD-SD']).sum()
print("Filas sin datos (SD):")
print(resultado[resultado > 0])

Filas sin datos (SD):
HORA                1
HH                  1
LUGAR_DEL_HECHO     1
PARTICIPANTES       5
VICTIMA             9
ACUSADO            23
dtype: int64


In [62]:
df_homicidios.duplicated().sum()

0

#### Transformacion de datos

Luego de examinar los valores sin datos de las columnas **Victima** y **Acusado**. Llegamos a la conclusion de que no es posible imputar estos datos, ya que el valor correspondiente de la columna **Participantes** tampoco cuenta con ese valor para reemplazarlo.

In [63]:
victimas = df_homicidios[df_homicidios['VICTIMA'] == 'SD']
victimas.sample(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
76,2016-0101,1,2016-08-07,2016,8,7,19:42:00,19,AV GRAL PAZ Y JORGE CHAVEZ,GRAL PAZ,...,NaN,"CHAVEZ, JORGE","PAZ, GRAL. AV. y CHAVEZ, JORGE",9,Point (93946.81673714 97681.68907836),-58.52932872,-34.65014993,SD-SD,SD,SD
269,2017-0155,1,2017-12-12,2017,12,12,21:25:41,21,AV. RIVADAVIA 6350,AVENIDA,...,6350.0,NaN,RIVADAVIA AV. 6350,7,Point (100674.98109200 100299.64949548),-58.45593915,-34.62656868,SD-MOTO,SD,MOTO
164,2017-0029,1,2017-03-07,2017,3,7,02:15:00,2,MOZART Y TANDIL,CALLE,...,NaN,TANDIL,MOZART y TANDIL,9,Point (98009.35575364 97985.81952007),-58.48501341,-34.64742455,SD-AUTO,SD,AUTO


In [64]:
acusado = df_homicidios[df_homicidios['ACUSADO'] == 'SD']
acusado.sample(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,...,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,SD,SD
216,2017-0091,1,2017-07-15,2017,7,15,13:00:00,13,SUPERI Y AV. GRAL. PAZ,GRAL PAZ,...,NaN,"PAZ, GRAL. AV.","SUPERI y PAZ, GRAL. AV.",12,Point (97850.57417090 109546.39428377),-58.48671535,-34.54321216,MOTO-SD,MOTO,SD
393,2018-0119,1,2018-11-05,2018,11,5,22:35:00,22,Rabanal y San Pedrito,AVENIDA,...,NaN,SAN PEDRITO,"RABANAL, FRANCISCO, INTENDENTE AV. y SAN PEDRITO",8,Point (102489.86457298 96266.15892101),-58.43613623,-34.66292532,PEATON-SD,PEATON,SD


In [65]:
#Esta funcion normaliza los nombre de columnas, pone solamente la primer letra en Mayuscula
def name_normalization(df):
    for name in df.columns:
      newName = name.capitalize() 
      df.rename(columns= {name:newName},inplace=True)

In [66]:
name_normalization(df_homicidios)

#### Eliminacion de columnas
Tras examinar las columnas del dataframe, podemos observar que hay columnas con valores nulos y columnas que no son necesarias para nuestro futuro analisis.

Por lo tanto la columna **Altura** sera eliminada debido a la gran cantidad de valores faltantes. Tanto la columna **Dirección Normalizada** y **Lugar del hecho** no las considero necesarias por lo tanto seran eliminadas. La columna **Cruce** la considero irrelevante, ya que contamos con la calle en la que ocurrio el accidente.
La columna **Hora** contiene la hora junto con los minutos en la que ocurrio el accidente, sera eliminada ya que contamos con la columna **Horario* la cual tiene la hora redondeada.

In [69]:
#Eliminacion de columna irrelevantes
df_homicidios.drop(['Altura','Hora','Dirección normalizada','Cruce','Lugar_del_hecho'],axis=1,inplace=True)

In [96]:
# Cambio de nombre las columnas
df_homicidios.rename(columns={'Id':'Id_hecho', 'N_victimas':'Num_victimas',
                          'Aaaa':'Año', 'Mm':'Mes', 
                          'Dd':'Dia', 'Hh':'Horario',
                          'Pos x':'Longitud', 'Pos y':'Latitud',},inplace=True)
df_homicidios.head(1)

,Id_hecho,Num_victimas,Fecha,Año,Mes,Dia_semana,Horario,Tipo_de_calle,Calle,Comuna,Longitud,Latitud,Participantes,Victima,Acusado,Coordenada_x,Coordenada_y
0,2016-0001,1,2016-01-01,2016,1,Viernes,4,AVENIDA,PIEDRA BUENA AV.,8.0,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378


Tratamiento de la columna ' **Xy (caba)**' 
- Se eliminan los datos que no sirven: "Point" "("  ")"
- Se separan las coordenadas, se transforman a float y se almacena cada coordenada en una lista para luego insertarlas en el dataframe como 2 columnas

In [83]:
# Elimino los valores entre [] de la columna
# Esto genera error           ----------->             (['Point', '(', ')'], '', regex=True)
df_homicidios['Xy (caba)'] = df_homicidios['Xy (caba)'].replace(['Point', r'\(', r'\)'], '', regex=True)

#Separo los valores de cada fila de la columna y los almaceno en una lista
lista_cordenadas = [cordenadas.split() for cordenadas in df_homicidios['Xy (caba)']]   
lista_cordenadas

# Listas para guardar cada valor
coordenadas_x = []
coordenadas_y = []

# Itera sobre cada elemento en la lista de coordenadas
for elemento in lista_cordenadas:
    
    if len(elemento) >= 2: #Si en la lista hay 2 o mas elementos        
        try: #Intenta convertir cada elemento en un float e insertalo en cada lista
            coordenadas_x.append(float(elemento[0]))
            coordenadas_y.append(float(elemento[1]))
        except ValueError:
            coordenadas_x.append(np.nan)
            coordenadas_y.append(np.nan)
           

In [ ]:
#Creo 2 nuevas columnas y les asignos las coredenadas extraidas anteriormente
df_homicidios['Coordenada_x'] = coordenadas_x
df_homicidios['Coordenada_y'] = coordenadas_y
#Elimino la columna 'Xy (caba)' porque ya no es necesario
df_homicidios.drop(['Xy (caba)'],axis=1,inplace=True)

In [ ]:
# Creo una la columna 'dia_semana' que contiene el nombre del día de la semana
df_homicidios['Dia_semana'] = df_homicidios['Fecha'].dt.day_name()
diccio={ 'Friday':'Viernes',
       'Saturday': 'Sabado',
       'Sunday': 'Domingo',  
       'Thursday': 'Jueves',
        'Monday': 'Lunes',
        'Wednesday': 'Miercoles',
       'Tuesday': 'Martes'}

df_homicidios['Dia_semana'] = df_homicidios['Dia_semana'].replace(diccio)
df_homicidios.drop(['Dia'],axis=1,inplace=True)

Reemplazo de valores

In [99]:
# Reemplaza los '.' por nulos en Longitud y Latitud.
df_homicidios['Longitud'].replace('.',np.nan, inplace=True)
df_homicidios['Latitud'].replace('.',np.nan, inplace=True)

# Reemplazamos los '0' por nulos en la columna Comuna
df_homicidios['Comuna'].replace(0,np.nan, inplace=True)

In [100]:
# Posicion de columnas.  17 = Dia_semana
new_order = [0,1,2,3,4,16,5,6,7,8,9,10,11,12,13,14,15]
# Reordenar las columnas
df_homicidios = df_homicidios.iloc[:, new_order]

In [101]:
df_homicidios.head(3)

,Id_hecho,Num_victimas,Fecha,Año,Mes,Coordenada_y,Dia_semana,Horario,Tipo_de_calle,Calle,Comuna,Longitud,Latitud,Participantes,Victima,Acusado,Coordenada_x
0,2016-0001,1,2016-01-01,2016,1,93532.434378,Viernes,4,AVENIDA,PIEDRA BUENA AV.,8.0,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,98896.782384
1,2016-0002,1,2016-01-02,2016,1,95505.416420,Sabado,1,GRAL PAZ,"PAZ, GRAL. AV.",9.0,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711
2,2016-0003,1,2016-01-03,2016,1,99706.576878,Domingo,7,AVENIDA,ENTRE RIOS AV.,1.0,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,106684.290900


In [102]:
#Guardo el DF en formato CSV
df_homicidios.to_csv('./datasets/df_homicidios.csv',index=False)

#### Extraccion 'victimas.xlsx'

In [103]:
df_victimas = pd.read_excel('Datasets/homicidios.xlsx',sheet_name='VICTIMAS')
df_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [104]:
datosNulos(df_victimas)

,Columna,Tipo de dato,Nulos,Porcentaje
0,ID_hecho,object,0,0 %
1,FECHA,datetime64[ns],0,0 %
2,AAAA,int64,0,0 %
3,MM,int64,0,0 %
4,DD,int64,0,0 %
5,ROL,object,0,0 %
6,VICTIMA,object,0,0 %
7,SEXO,object,0,0 %
8,EDAD,object,0,0 %
9,FECHA_FALLECIMIENTO,object,0,0 %


In [105]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [106]:
#Elimino las columnas que considero innecesarias.
df_victimas.drop(['AAAA','MM','DD','FECHA_FALLECIMIENTO'],axis=1,inplace=True)
 
# df_victimas.rename(columns= {'Aaaa':'Año','Mm':'Mes','Dd':'Dia'},inplace=True)

In [107]:
name_normalization(df_victimas)

In [108]:
#Muestra las columnas sin datos
count_SD = df_victimas.apply(lambda x: x.value_counts().get('SD', 0))
count_SD

Id_hecho     0
Fecha        0
Rol         11
Victima      9
Sexo         6
Edad        53
dtype: int64

In [109]:
#Reemplazo los valores 'SD' por Nan en la columna Edad
df_victimas.loc[df_victimas['Edad'] == 'SD', 'Edad'] = np.nan
df_victimas['Edad'] = df_victimas['Edad'].astype(float)

#df_victimas['Edad'].astype(int) No se puede convertir a int porque hay valores Nan

In [110]:
df_victimas['Edad'].unique()

array([19., 70., 30., 18., 29., 22., 16., 59., 65., 34., 41., 50., 38.,
       21., 52., 36., 20., 54., nan, 56., 24., 78., 79., 26., 57., 37.,
       58., 23., 60., 42., 53., 51., 40., 87., 76., 75., 35., 80., 43.,
       45., 67., 27., 55., 49., 81., 25., 33., 46., 83., 39., 28.,  7.,
       48.,  4., 82., 32., 17., 47., 61., 10., 95., 73., 84., 66., 85.,
        1., 15., 13., 77., 44., 31., 62., 74., 71., 11., 86., 69., 72.,
       12., 63., 92., 68., 91., 64.,  5., 88.])

In [111]:
rangos = [0, 20, 35, 45, 59, 100]
#Etiquetas son los valores que contendra la columna
etiquetas = ['Menor de 21', 'Entre 21 y 35', 'Entre 36 y 45', 'Entre 46 y 59', '60 o mas']

# Crear la columna 'Rango_etario' utilizando pd.cut()
df_victimas['Rango_etario'] = pd.cut(df_victimas['Edad'], bins=rangos, labels=etiquetas)
    

In [112]:
df_victimas.head()

,Id_hecho,Fecha,Rol,Victima,Sexo,Edad,Rango_etario
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19.0,Menor de 21
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70.0,60 o mas
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30.0,Entre 21 y 35
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18.0,Menor de 21
4,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29.0,Entre 21 y 35


In [17]:
df_victimas.to_csv('./datasets/df_victimas.csv',index=False)